# Example: Importance Sampling
The J/Psi -> gamma pi0 pi0 decay is used here (which has a narrow omega state)

Let's go!

## Step 1: Define intensity
First we create the intensity (If you do not know this step, check other amplitude construting examples using the expert system).

In [ ]:
from expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from expertsystem.amplitude.helicitydecay import (
    HelicityDecayAmplitudeGeneratorXML)
from expertsystem.topology.graph import (
    get_intermediate_state_edges)
from expertsystem.state.particle import (
    get_particle_with_name, particle_list)
    
initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma", [-1, 1]), ("pi0", [0]), ("pi0", [0])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

# The omega is so narrow that the hit&miss generation takes way too long.
# Therefore we increase the width artificially in this example!
omega = get_particle_with_name('omega')
parameters = omega['DecayInfo']['Parameter']
for par in parameters:
    if par['@Type'] == 'Width':
        par['Value'] = 0.001

tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
tbd_manager.allowed_intermediate_particles = ['f2(1270)', 'omega']
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")

xml_generator = HelicityDecayAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

## Step 2: Create intensity and generate importance weighted phase space sample
Now we generate a phase space sample which is importance sampled by the intensity

In [ ]:
# pycompwa is the python interface to ComPWA's c++ modules
import pycompwa as pwa

# Create particle list
particle_list = pwa.PartList()
with open('model.xml', 'r') as content_file:
    model_file_contents = content_file.read()
    pwa.read_particles(particle_list, model_file_contents)

# Create kinematics
kin = pwa.HelicityKinematics(particle_list, 'model.xml')

# Generate phase space sample (flat) used for amplitude normalization
gen = pwa.EvtGenGenerator(particle_list, kin, 12345)
phsp_sample = pwa.generate_phsp(100000, gen)

# Create Amplitude
with open('model.xml', 'r') as content_file:
    model_file_contents = content_file.read()
    intensity = pwa.incoherent_intensity(model_file_contents, 
                                         particle_list,
                                         kin, phsp_sample,
                                         phsp_sample)

# Generate importance sampled phase space sample
phsp_sample_importance = pwa.generate_importance_sampled_phsp(1000, kin, gen, intensity)


# Generate Data
#sample = pwa.generate(1000, kin, gen, intensity)

## Step 3: Visualize the phase space sample
Let's go ahead and make a Dalitz plot of this phase space sample. We use the direct access of the python interface to the data points!

In [ ]:
# create all subsystems 
#(since we have both f and the omega resonances, all subsystems already exist and this
#step is redundant)
kin.create_all_subsystems()

import numpy as np
# use the direct data point access via DataPoints
data_points = pwa.DataPoints(phsp_sample_importance, kin)
data_array = np.array(data_points)
variable_names = data_points.get_variable_names()
print(variable_names)

# check that the sum of weights is equal to the number of events (should be 1000)!
print("sum of weights (should be 1000):", sum(data_array[:,variable_names.index("weight")]))

# Plotting
from Plotting.plot import (
    make_dalitz_plots, PlotData
)

plotdata = PlotData(column_names=variable_names, data_array=data_array)
plotdata.particle_id_to_name_mapping = data_points.get_finalstate_id_to_name_mapping()
# plot a 2d histogram
make_dalitz_plots(plotdata, ["mSq_3_4_vs_2", "mSq_2_4_vs_3"], bins=50)